In [25]:
import numpy as np
import cv2 as cv2
import tensorflow as tf
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline
tf.compat.v1.disable_eager_execution()

In [26]:
# 获取文件夹的数据

def get_imgs(rate = 0.2):
    """
    获取图片，并划分训练集和测试集
    Parameters:
        rate:测试集 2 和训练集 10 的比例，即测试集个数/训练集个数
    Returns:
        test_imgs:测试集
        test_labels:测试集标签
        train_imgs:训练集
        train_labels:训练集标签
    """
    # 读取图片
    imgs = os.listdir('./verify/')
    # 打乱图片顺序
    random.shuffle(imgs)
    # 数据集总共个数
    imgs_num = len(imgs)
    # 按照比例求出测试集个数
    test_num = int(imgs_num * rate / (1 + rate))
    # 测试集，测试数据的路径
    test_imgs = imgs[:test_num]
    # 根据文件名获取测试集标签
    test_labels = list(map(lambda x: x.split('.')[0], test_imgs))
    # 训练集
    train_imgs = imgs[test_num:]
    # 根据文件名获取训练集标签
    train_labels = list(map(lambda x: x.split('.')[0], train_imgs))

    return test_imgs, test_labels, train_imgs, train_labels

In [27]:
# 定义了每次取出一批数据，tensorflow训练的时候，一批批喂给算法，for循环执行很多次

def get_next_batch(test_imgs,test_labels,train_imgs,train_labels,train_flag=True, batch_size=100):

    train_size = 4160 # 训练数据的长度
    test_size = 831   # 测试数据的长度
    train_ptr = 0    # 从0开始取值
    test_ptr = 0
    height = 30 # 图片的高度
    width = 100 # 图片的宽度
    max_captcha = 4 # 最大的验证码长度
    # 0~ 9（10），a~z（26） ，A~Z（26） --------> 62 + 1 (_未知) -----> 63
    char_set_len = 63 # 所有的字符数
    data_path = './verify/' # 数据的路径
    """
    获得batch_size大小的数据集
    Parameters:
        batch_size:batch_size大小
        train_flag:是否从训练集获取数据
    Returns:
        batch_x:大小为batch_size的数据x
        batch_y:大小为batch_size的数据y
        image(用于测试)
    """
    # 从训练集获取数据
    if train_flag == True:
        if (batch_size + train_ptr) < train_size:
            trains = train_imgs[train_ptr:(train_ptr + batch_size)]
            labels = train_labels[train_ptr:(train_ptr + batch_size)]
            train_ptr += batch_size
        else:
            new_ptr = (train_ptr + batch_size) % train_size
            trains = train_imgs[train_ptr:] + train_imgs[:new_ptr]
            labels = train_labels[train_ptr:] + train_labels[:new_ptr]
            train_ptr = new_ptr

#       返回数据，给了形状
        batch_X = np.zeros([batch_size, height*width])
#       目标值，独热编码表示 4 * 63 ------> 概率问题
        batch_y = np.zeros([batch_size, max_captcha*char_set_len])

        for index, train in enumerate(trains):
            # 黑白图片
            img = np.mean(cv2.imread(data_path + train), axis = -1)
            # 将多维降维1维
            batch_X[index,:] = img.flatten() / 255 # flatten()   ---> reshape(-1)  ---- > ravel()
        for index, label in enumerate(labels):
            batch_y[index,:] = text2vec(label)

    # 从测试集获取数据
    else:
        if (batch_size + test_ptr) < test_size:
            tests = test_imgs[test_ptr:(test_ptr + batch_size)]
            labels = test_labels[test_ptr:(test_ptr + batch_size)]
            test_ptr += batch_size
        else:
            new_ptr = (test_ptr + batch_size) % test_size
            tests = test_imgs[test_ptr:] + test_imgs[:new_ptr]
            labels = test_labels[test_ptr:] + test_labels[:new_ptr]
            test_ptr = new_ptr

        batch_X = np.zeros([batch_size, height*width])
        batch_y = np.zeros([batch_size, max_captcha*char_set_len])

        for index, test in enumerate(tests):
#             图片灰度化处理，黑白处理
            img = np.mean(cv2.imread(data_path + test), axis = -1)
            # 将多维降维1维
            batch_X[index,:] = img.ravel() / 255
        for index, label in enumerate(labels):
            batch_y[index,:] = text2vec(label)
        return batch_X, batch_y
    return batch_X, batch_y

In [28]:
def vec2text(vec):
    char_set_len = 63
    """
    向量转文本
    Parameters:
        vec:向量
    Returns:
        文本
    """
    char_pos = vec.nonzero()[0]
    text = []
    for c in char_pos:
        char_idx = c % char_set_len
        if char_idx < 10:
            char_code = char_idx + ord('0')
        elif char_idx < 36:
            char_code = char_idx - 10 + ord('A')
        elif char_idx < 62:
            char_code = char_idx - 36 + ord('a')
        elif char_idx == 62:
            char_code = ord('_')
        else:
            raise ValueError('error')
        text.append(chr(char_code))
    return "".join(text)

In [29]:
def text2vec(text):
    char_set_len = 63

    """
    文本转向量
    Parameters:
        text:文本
    Returns:
        vector:向量
    """
    if len(text) > 4:
        raise ValueError('验证码最长4个字符')

    vector = np.zeros(4 * char_set_len)
    def char2pos(c):
        if c =='_':
            k = 62
            return k
        k = ord(c) - 48
        if k > 9:
            k = ord(c) - 55
            if k > 35:
                k = ord(c) - 61
                if k > 61:
                    raise ValueError('No Map')
        return k
    for i, c in enumerate(text):
        idx = i * char_set_len + char2pos(c)
        vector[idx] = 1
    return vector

In [30]:
X=tf.compat.v1.placeholder(dtype=tf.float64,shape=[None,30*100])
y=tf.compat.v1.placeholder(dtype=tf.float64,shape=[None,4*63])
kp=tf.compat.v1.placeholder(dtype=tf.float64,shape=None)

def crack_captcha_cnn():
    # 第一层神经网络(卷积)
    
    input_data= tf.reshape(X,shape=[-1,30,100,1])
    filters1=tf.Variable(initial_value=tf.random.normal(dtype=tf.float64,shape=[3,3,1,32],stddev=0.01),
                         dtype=tf.float64)
    # filters 为32个方程 需要32个b1
    b1=tf.Variable(initial_value=tf.random.normal(dtype=tf.float64,shape=[32],stddev=0.01),dtype=tf.float64)
    
    conv1=tf.nn.conv2d(input=input_data,filters=filters1,strides=[1,1,1,1],padding="SAME")+b1
    # 激活函数
    relu1=tf.nn.relu(conv1)
    
    # 池化(选择特征 特性提取) strides 步幅
     
    pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    # pool1.shape   ---->  [-1,50,100,32]
    # 第二层神经网络(卷积)
    # 输入32 输出64
    filters2=tf.Variable(initial_value=tf.random.normal(dtype=tf.float64,shape=[3,3,32,64],stddev=0.01),
                         dtype=tf.float64)
    
    b2=tf.Variable(initial_value=tf.random.normal(dtype=tf.float64,shape=[64],stddev=0.01),dtype=tf.float64)
    conv2=tf.nn.conv2d(input=pool1,filters=filters2,strides=[1,1,1,1],padding="SAME")+b2

    # 激活函数
    sigmoid1=tf.nn.sigmoid(conv2)
    # 池化(选择特征 特性提取) strides 步幅
    pool2=tf.nn.max_pool(sigmoid1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    
    # pool2.shape ----> [-1,8,25,64]

    # 第三层神经网络(卷积)

    # 输入64 输出128
    filters3=tf.Variable(initial_value=tf.random.normal(dtype=tf.float64,shape=[3,3,64,128],stddev=0.01)
                         ,dtype=tf.float64)
    
    b3=tf.Variable(initial_value=tf.random.normal(dtype=tf.float64,shape=[128],stddev=0.01),dtype=tf.float64)
    
    conv3=tf.nn.conv2d(input=pool2,filters=filters3,strides=[1,1,1,1],padding="SAME")+b3
    # 激活函数
    relu=tf.nn.relu(conv3)
    # 池化(选择特征 特性提取) strides 步幅
    pool3=tf.nn.max_pool(relu,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    
    # pool3.shape -----> [-1,4,13,128 ]
    
    # 全连接层 1024个神精元
    dense=tf.reshape(pool3,shape=[-1,4*13*128])
    
    conn_w=tf.Variable(
        dtype=tf.float64,
        initial_value=tf.random.normal(shape=[4*13*128,1024],dtype=tf.float64,stddev=0.01)
    )
    conn_b=tf.Variable(
        dtype=tf.float64,
        initial_value=tf.random.normal(shape=[1024],dtype=tf.float64,stddev=0.01)
    )
    conn=tf.matmul(dense,conn_w)+conn_b
    # conn.shape ------> [-1,1024]
    
    # 激活函数
    conn_relu=tf.nn.relu(conn)
    
    # dropout 防止过拟合
    dropout=tf.nn.dropout(conn_relu,rate=kp)
    # 输出层 y进行对比
    out_w=tf.Variable(
        initial_value=tf.random.normal(dtype=tf.float64,shape=[1024,4*63],stddev=0.01),
        dtype=tf.float64,
    )
    out_b=tf.Variable(
        initial_value=tf.random.normal(dtype=tf.float64,shape=[4*63],stddev=0.01),
        dtype=tf.float64,
    )
    out=tf.matmul(dropout,out_w)+out_b
    # out.shahpe ------> [?,4*64]
    return out 

In [31]:
crack_captcha_cnn()

<tf.Tensor 'add_24:0' shape=(None, 252) dtype=float64>

In [32]:
# 训练
#     tf.compat.v1.train.Saver()

def train_crack_captcha_cnn():
    saver=tf.compat.v1.train.Saver()

    out=crack_captcha_cnn() # 代码预测出来的数据
#     prob_=tf.nn.softmax(out) # 转化成概率 预测的概率非真实分布
    cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=out))
    
    
    optimizer=tf.compat.v1.train.AdamOptimizer().minimize(cost) #  自使用梯度下降
    # 定义计算正确率的
    prob_=tf.nn.softmax(out)
    # accuracy type: Tensor
    accuracy =tf.reduce_mean(tf.cast(tf.equal(x=tf.argmax(prob_,axis=-1),y=tf.argmax(y,axis=-1)),dtype=tf.float64))
    
    
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)  
    config_=tf.compat.v1.ConfigProto(log_device_placement=True,allow_soft_placement=True)

    with tf.compat.v1.Session(config=config_) as sess:
#     with tf.compat.v1.Session() as sess:
    
        sess.run(tf.compat.v1.global_variables_initializer())
        test_imgs, test_labels, train_imgs, train_labels=get_imgs(rate=0.2)
        for i in range(100000):
            if (i==500)&(i!=0):
                test_imgs, test_labels, train_imgs, train_labels=get_imgs(rate=0.2)
            X_train,y_train=get_next_batch(test_imgs, test_labels, train_imgs, train_labels)
            optimizer_,cost_=sess.run(fetches=[optimizer,cost],feed_dict={X:X_train,y:y_train,kp:0.5})
            if i %100==0:
                X_test,y_test=get_next_batch(test_imgs, test_labels, train_imgs, train_labels,train_flag=False,batch_size=500)
                score=sess.run(fetches=accuracy,feed_dict={X:X_test,y:y_test,kp:1})
                print('算法执行次数：{}算法准确率：{}'.format(i,score))
                if score>=0.96: 
                    saver.save(sess,"./captcha/model",i)
            if (i %1000==0)&(i!=0):
                saver.save(sess,"./captcha/model",i)

In [33]:
# train_crack_captcha_cnn()

In [34]:
# 训练
#     tf.compat.v1.train.Saver()
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


def train_captcha_cnn():
    saver=tf.compat.v1.train.Saver()
    out=crack_captcha_cnn() # 代码预测出来的数据
#     prob_=tf.nn.softmax(out) # 转化成概率 预测的概率非真实分布
    cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=out))
    
    
    optimizer=tf.compat.v1.train.AdamOptimizer().minimize(cost) #  自使用梯度下降
    # 定义计算正确率的
    # accuracy type: Tensor
    
    config = ConfigProto()
    config.gpu_options.allow_growth = True
#     gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)  
#     config_=tf.compat.v1.ConfigProto(log_device_placement=True,allow_soft_placement=True)
    with InteractiveSession(config=config) as sess:    
        sess.run(tf.compat.v1.global_variables_initializer())
        test_imgs, test_labels, train_imgs, train_labels=get_imgs(rate=0.2)
        for i in range(1000):
            X_train,y_train=get_next_batch(test_imgs, test_labels, train_imgs, train_labels)
            optimizer_,cost_=sess.run(fetches=[optimizer,cost],feed_dict={X:X_train,y:y_train,kp:0.5})
            if i %100==0:
                    saver.save(sess,"./captcha/model",i)

In [35]:
train_captcha_cnn()

d:\python\venv(data_analysis)\lib\site-packages\tensorflow\python\client\session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


AttributeError: __enter__

In [ ]:
test_imgs, test_labels, train_imgs, train_labels=get_imgs(rate=0.2)

In [ ]:
batch_X, batch_y=get_next_batch(test_imgs, test_labels, train_imgs, train_labels)

In [ ]:
vec=text2vec("ac2l")
vec


In [ ]:
vec.nonzero()[0]


In [ ]:
test_imgs, test_labels, train_imgs, train_labels=get_imgs()
test_imgs[100]
image=plt.imread("./verify/"+test_imgs[100])
image.shape